# Creating datasets

Creating datasets with PyTables via NumPy arrays is easy:

In [1]:
import numpy as np
import tables as tb

# Create a new file
f = tb.open_file("atest.h5", "w")

# Create a NumPy array
a = np.arange(100).reshape(20,5)

# Save the array
f.create_array(f.root, "array1", a)

/array1 (Array(20, 5)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [2]:
# Peek data
f.root.array1

/array1 (Array(20, 5)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [3]:
# Slice and dice (only these slices are loaded into memory)
ta = f.root.array1
ta[1:10:3,2:5]

array([[ 7,  8,  9],
       [22, 23, 24],
       [37, 38, 39]])

In [4]:
# Make sure that the read data is the same than the original
np.allclose(ta[1:10:3,2:5], a[1:10:3,2:5])

True

In [5]:
# Create another array
ta2 = f.create_array(f.root, "array2", np.arange(10))

In [6]:
# Let's have a look at the size of the underlying file
!ls -l atest.h5

-rw-rw-r-- 1 francesc francesc 0 Oct 21 15:27 atest.h5


In [7]:
# Flush data to the file (very important to keep all your data safe!)
f.flush()

In [8]:
!ls -l atest.h5

-rw-rw-r-- 1 francesc francesc 3024 Oct 21 15:27 atest.h5


In [9]:
f.close()  # close access to file

In [10]:
# Look at its contents by using `ptdump` utility
! ptdump atest.h5

/ (RootGroup) ''
/array1 (Array(20, 5)) ''
/array2 (Array(10,)) ''


In [11]:
# Reopen the file and revisit the datasets there
f = tb.open_file("atest.h5", mode="r")  # note the 'r'ead mode

In [12]:
# Get the summary of the contents
print(f)

atest.h5 (File) ''
Last modif.: 'Fri Oct 21 15:27:23 2016'
Object Tree: 
/ (RootGroup) ''
/array1 (Array(20, 5)) ''
/array2 (Array(10,)) ''



In [13]:
f.root.array1

/array1 (Array(20, 5)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [14]:
f.root.array1[:]

array([[ 0,  1,  2,  3,  4],
       [ 5,  6,  7,  8,  9],
       [10, 11, 12, 13, 14],
       [15, 16, 17, 18, 19],
       [20, 21, 22, 23, 24],
       [25, 26, 27, 28, 29],
       [30, 31, 32, 33, 34],
       [35, 36, 37, 38, 39],
       [40, 41, 42, 43, 44],
       [45, 46, 47, 48, 49],
       [50, 51, 52, 53, 54],
       [55, 56, 57, 58, 59],
       [60, 61, 62, 63, 64],
       [65, 66, 67, 68, 69],
       [70, 71, 72, 73, 74],
       [75, 76, 77, 78, 79],
       [80, 81, 82, 83, 84],
       [85, 86, 87, 88, 89],
       [90, 91, 92, 93, 94],
       [95, 96, 97, 98, 99]])

In [15]:
# Always close your files when you are done (or use contexts)
f.close()

## Exercise

Create a new HDF5 file with 2 arrays on it.  One should be 2-dimensional and the other the result of summing the 2nd dimension (.sum(axis=1)).  Use contexts so that you don't have to close the file explicitly.

### Solution

In [16]:
arr2d = np.arange(100, dtype=np.int32).reshape(10, 10)
with tb.open_file("myfile.h5", "w") as f:
    f.create_array(f.root, "arr2d", arr2d)
    f.create_array(f.root, "sum1d", arr2d.sum(axis=1))
! ptdump -d myfile.h5

/ (RootGroup) ''
/arr2d (Array(10, 10)) ''
  Data dump:
[0] [0 1 2 3 4 5 6 7 8 9]
[1] [10 11 12 13 14 15 16 17 18 19]
[2] [20 21 22 23 24 25 26 27 28 29]
[3] [30 31 32 33 34 35 36 37 38 39]
[4] [40 41 42 43 44 45 46 47 48 49]
[5] [50 51 52 53 54 55 56 57 58 59]
[6] [60 61 62 63 64 65 66 67 68 69]
[7] [70 71 72 73 74 75 76 77 78 79]
[8] [80 81 82 83 84 85 86 87 88 89]
[9] [90 91 92 93 94 95 96 97 98 99]
/sum1d (Array(10,)) ''
  Data dump:
[0] 45
[1] 145
[2] 245
[3] 345
[4] 445
[5] 545
[6] 645
[7] 745
[8] 845
[9] 945


# Playing with the object tree

In [17]:
# Re-open the existing file in 'a'ppend mode
f = tb.open_file("atest.h5", "a")

In [18]:
f

File(filename=atest.h5, title='', mode='a', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/array1 (Array(20, 5)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/array2 (Array(10,)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [19]:
# We can get a shortened view too:
print(f)

atest.h5 (File) ''
Last modif.: 'Fri Oct 21 15:27:23 2016'
Object Tree: 
/ (RootGroup) ''
/array1 (Array(20, 5)) ''
/array2 (Array(10,)) ''



In [20]:
# Add a new group
f.create_group(f.root, 'group1', 'Title for group1')

/group1 (Group) 'Title for group1'
  children := []

In [21]:
f

File(filename=atest.h5, title='', mode='a', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/array1 (Array(20, 5)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/array2 (Array(10,)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/group1 (Group) 'Title for group1'

In [22]:
f.move_node(f.root.array1, f.root.group1)

In [23]:
f

File(filename=atest.h5, title='', mode='a', root_uep='/', filters=Filters(complevel=0, shuffle=False, bitshuffle=False, fletcher32=False, least_significant_digit=None))
/ (RootGroup) ''
/array2 (Array(10,)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None
/group1 (Group) 'Title for group1'
/group1/array1 (Array(20, 5)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [24]:
# Create a very nested group (note the `createparents` parameter)
f.create_group('/g1/g2/g3/g4', 'g5', createparents=True)

/g1/g2/g3/g4/g5 (Group) ''
  children := []

In [25]:
print(f)

atest.h5 (File) ''
Last modif.: 'Fri Oct 21 15:27:23 2016'
Object Tree: 
/ (RootGroup) ''
/array2 (Array(10,)) ''
/g1 (Group) ''
/group1 (Group) 'Title for group1'
/group1/array1 (Array(20, 5)) ''
/g1/g2 (Group) ''
/g1/g2/g3 (Group) ''
/g1/g2/g3/g4 (Group) ''
/g1/g2/g3/g4/g5 (Group) ''



In [26]:
# Add an array in the 'very nested' group
f.create_array(f.root.g1.g2.g3.g4.g5, 'array2', np.arange(10))

/g1/g2/g3/g4/g5/array2 (Array(10,)) ''
  atom := Int64Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := None

In [27]:
print(f)

atest.h5 (File) ''
Last modif.: 'Fri Oct 21 15:27:23 2016'
Object Tree: 
/ (RootGroup) ''
/array2 (Array(10,)) ''
/g1 (Group) ''
/group1 (Group) 'Title for group1'
/group1/array1 (Array(20, 5)) ''
/g1/g2 (Group) ''
/g1/g2/g3 (Group) ''
/g1/g2/g3/g4 (Group) ''
/g1/g2/g3/g4/g5 (Group) ''
/g1/g2/g3/g4/g5/array2 (Array(10,)) ''



In [28]:
# Removing nodes is very easy
f.remove_node(f.root.g1.g2.g3.g4.g5.array2)

In [29]:
print(f)

atest.h5 (File) ''
Last modif.: 'Fri Oct 21 15:27:24 2016'
Object Tree: 
/ (RootGroup) ''
/array2 (Array(10,)) ''
/g1 (Group) ''
/group1 (Group) 'Title for group1'
/group1/array1 (Array(20, 5)) ''
/g1/g2 (Group) ''
/g1/g2/g3 (Group) ''
/g1/g2/g3/g4 (Group) ''
/g1/g2/g3/g4/g5 (Group) ''



In [30]:
# Show the PyTables File object working as an iterator
for n in f:
    print(n)

/ (RootGroup) ''
/array2 (Array(10,)) ''
/g1 (Group) ''
/group1 (Group) 'Title for group1'
/g1/g2 (Group) ''
/group1/array1 (Array(20, 5)) ''
/g1/g2/g3 (Group) ''
/g1/g2/g3/g4 (Group) ''
/g1/g2/g3/g4/g5 (Group) ''


In [31]:
# The `File.walk_nodes` method offers more flexibility
for n in f.walk_nodes():
    print(n)

/ (RootGroup) ''
/array2 (Array(10,)) ''
/g1 (Group) ''
/group1 (Group) 'Title for group1'
/g1/g2 (Group) ''
/group1/array1 (Array(20, 5)) ''
/g1/g2/g3 (Group) ''
/g1/g2/g3/g4 (Group) ''
/g1/g2/g3/g4/g5 (Group) ''


In [32]:
# Get info from a certain point of the hierarchy on
for n in f.walk_nodes(f.root.group1):
    print(n)

/group1 (Group) 'Title for group1'
/group1/array1 (Array(20, 5)) ''


In [33]:
# walknodes allows to iterate over specific classes
for n in f.walk_nodes(f.root.group1, classname="Array"):
    print(n[:2])

[[0 1 2 3 4]
 [5 6 7 8 9]]


In [34]:
f.close()

## Exercise

Use the file that you created in the previous exercise and create a new group called 'reduced' and titled 'My Reduced data' and move the 1-dimensional array there.  Look at the final contents with the ptdump utility.  

### Solution

In [35]:
with tb.open_file("myfile.h5", "a") as f:
    f.create_group(f.root, "reduced", "My reduced data")
    f.move_node(f.root.sum1d, f.root.reduced)
! ptdump myfile.h5

/ (RootGroup) ''
/arr2d (Array(10, 10)) ''
/reduced (Group) 'My reduced data'
/reduced/sum1d (Array(10,)) ''


# HDF5 atributes

In [36]:
# Re-open the file
f = tb.open_file("atest.h5", "a")

In [37]:
print(f)

atest.h5 (File) ''
Last modif.: 'Fri Oct 21 15:27:24 2016'
Object Tree: 
/ (RootGroup) ''
/array2 (Array(10,)) ''
/g1 (Group) ''
/group1 (Group) 'Title for group1'
/group1/array1 (Array(20, 5)) ''
/g1/g2 (Group) ''
/g1/g2/g3 (Group) ''
/g1/g2/g3/g4 (Group) ''
/g1/g2/g3/g4/g5 (Group) ''



In [38]:
# Print the attrs in /array2
f.root.array2.attrs

/array2._v_attrs (AttributeSet), 4 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4']

In [39]:
# Add a new attribute to /array2
f.root.array2.attrs.myattr = "Hello World!"

In [40]:
f.root.array2.attrs

/array2._v_attrs (AttributeSet), 5 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    myattr := 'Hello World!']

In [41]:
# Has the modification arrived to disk yet? 
!ptdump -a atest.h5:/array2  # note the -a flag and node specification

/array2 (Array(10,)) ''
  /array2._v_attrs (AttributeSet), 4 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4']


In [42]:
# Nope, so force a flush
f.flush()

In [43]:
!ptdump -a atest.h5:/array2

/array2 (Array(10,)) ''
  /array2._v_attrs (AttributeSet), 5 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    myattr := 'Hello World!']


In [44]:
f.root._v_attrs.mydata = "My data file"
f.root._v_attrs

/._v_attrs (AttributeSet), 5 attributes:
   [CLASS := 'GROUP',
    PYTABLES_FORMAT_VERSION := '2.1',
    TITLE := '',
    VERSION := '1.0',
    mydata := 'My data file']

In [45]:
# Attributes can also be general arrays
f.root.array2.attrs.myarray = np.arange(10)
f.flush()

In [46]:
!ptdump -a atest.h5:/array2

/array2 (Array(10,)) ''
  /array2._v_attrs (AttributeSet), 6 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    myarray := array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
    myattr := 'Hello World!']


In [47]:
# Get a shortcut to the attribute handler
attrs = f.root.array2.attrs

In [48]:
attrs

/array2._v_attrs (AttributeSet), 6 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    myarray := array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
    myattr := 'Hello World!']

In [49]:
# Removing an attribute
del attrs.myarray
attrs

/array2._v_attrs (AttributeSet), 5 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    myattr := 'Hello World!']

In [50]:
# Overwrite an existing one (be careful with this feature!)
attrs.myattr = 12.3
attrs

/array2._v_attrs (AttributeSet), 5 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    myattr := 12.300000000000001]

In [51]:
# Print the attributes for all the arrays in the object tree
for n in f.walk_nodes(f.root.group1, classname="Array"):
    print(repr(n.attrs))

/group1/array1._v_attrs (AttributeSet), 4 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4']


In [52]:
f.close()

## Exercise

Use the file in the previous exercise and add an attribute to the 1-dimensional array specifying the mean and the standard deviation. Use ptdump -a to check that the attributes are there.

### Solution

In [53]:
with tb.open_file("myfile.h5", "a") as f:
    sum1d = f.root.reduced.sum1d
    arr = sum1d[:]
    sum1d.attrs.mean = arr.mean()
    sum1d.attrs.std = arr.std()
! ptdump -a myfile.h5:/reduced/sum1d

/reduced/sum1d (Array(10,)) ''
  /reduced/sum1d._v_attrs (AttributeSet), 6 attributes:
   [CLASS := 'ARRAY',
    FLAVOR := 'numpy',
    TITLE := '',
    VERSION := '2.4',
    mean := 495.0,
    std := 287.22813232690146]


# Chunked datasets

In [54]:
f = tb.open_file('ctest.h5', 'w')

In [55]:
# Create an un-initialized CArray (Compressible Array)
f.create_carray(f.root, 'carray', tb.Float64Atom(), (10000,1000))

/carray (CArray(10000, 1000)) ''
  atom := Float64Atom(shape=(), dflt=0.0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'little'
  chunkshape := (16, 1000)

In [56]:
# Flush everything to disk
f.flush()

In [57]:
# The container is there, but not the data (yet)
!ls -lh ctest.h5

-rw-rw-r-- 1 francesc francesc 1.7K Oct 21 15:27 ctest.h5


In [58]:
# Push some data into this carray container
ca = f.root.carray
na = np.linspace(0, 1, 1e7).reshape(10000,1000)
%time ca[:] = na

CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 23.7 ms


In [59]:
# Flush (we can specify which node should be flushed)
ca.flush()

In [60]:
!ls -lh ctest.h5

-rw-rw-r-- 1 francesc francesc 77M Oct 21 15:27 ctest.h5


In [61]:
np.prod(ca.shape) * ca.dtype.itemsize / 2**20.

76.2939453125

In [62]:
# Retrieve only part of the data
ca[:10,::2]

array([[  0.00000000e+00,   2.00000020e-07,   4.00000040e-07, ...,
          9.94000099e-05,   9.96000100e-05,   9.98000100e-05],
       [  1.00000010e-04,   1.00200010e-04,   1.00400010e-04, ...,
          1.99400020e-04,   1.99600020e-04,   1.99800020e-04],
       [  2.00000020e-04,   2.00200020e-04,   2.00400020e-04, ...,
          2.99400030e-04,   2.99600030e-04,   2.99800030e-04],
       ..., 
       [  7.00000070e-04,   7.00200070e-04,   7.00400070e-04, ...,
          7.99400080e-04,   7.99600080e-04,   7.99800080e-04],
       [  8.00000080e-04,   8.00200080e-04,   8.00400080e-04, ...,
          8.99400090e-04,   8.99600090e-04,   8.99800090e-04],
       [  9.00000090e-04,   9.00200090e-04,   9.00400090e-04, ...,
          9.99400100e-04,   9.99600100e-04,   9.99800100e-04]])

In [63]:
f.close()

## Using compression with chunked arrays

In [64]:
f = tb.open_file('ctest-zlib.h5', 'w')

In [65]:
# Create a CArray (Compressible Array) using the zlib compressor
filters = tb.Filters(complib='zlib', complevel=1, shuffle=True)
ca = f.create_carray(f.root, 'carray', tb.Float64Atom(), (10000,1000),
                     filters=filters)

In [66]:
# Push some data on this carray container
na = np.linspace(0, 1, 1e7).reshape(10000,1000)
%time ca[:] = na

CPU times: user 288 ms, sys: 4 ms, total: 292 ms
Wall time: 288 ms


In [67]:
# Flush the carray container only
ca.flush()
!ls -lh ctest-zlib.h5

-rw-rw-r-- 1 francesc francesc 6.8M Oct 21 15:27 ctest-zlib.h5


In [68]:
np.prod(ca.shape) * ca.dtype.itemsize / 2**20.

76.2939453125

In [69]:
f.close()

In [70]:
# Look at the file with a native HDF5 tool
!h5ls -v ctest-zlib.h5

Opened "ctest-zlib.h5" with sec2 driver.
carray                   Dataset {10000/10000, 1000/1000}
    Attribute: CLASS scalar
        Type:      6-byte null-terminated ASCII string
        Data:  "CARRAY"
    Attribute: TITLE null
        Type:      1-byte null-terminated ASCII string

    Attribute: VERSION scalar
        Type:      3-byte null-terminated ASCII string
        Data:  "1.1"
    Location:  1:1024
    Links:     1
    Chunks:    {16, 1000} 128000 bytes
    Storage:   80000000 logical bytes, 7004415 allocated bytes, 1142.14% utilization
    Filter-0:  shuffle-2 OPT {8}
    Filter-1:  deflate-1 OPT {1}
    Type:      native double
H5tools-DIAG: Error detected in HDF5:tools (1.8.17) thread 0:
  #000: h5tools_dump.c line 1836 in h5tools_dump_mem(): H5Sis_simple failed
    major: Failure in tools library
    minor: error in function


## Using compression (Blosc)

In [71]:
f = tb.open_file('ctest-blosc.h5', 'w')

In [72]:
# Create a CArray (Compressible Array) using the Blosc compressor
filters = tb.Filters(complib='blosc:lz4', complevel=9)
ca = f.create_carray(f.root, 'carray', tb.Float64Atom(), (10000,1000),
                     filters=filters)

In [73]:
# Push some data on this carray container
na = np.linspace(0, 1, 1e7).reshape(10000,1000)
%time ca[:] = na

CPU times: user 52 ms, sys: 4 ms, total: 56 ms
Wall time: 55.6 ms


**Note how witing a compressed carray is close in the this case to the uncompressed case above (~20 ms) and much faster than using zlib (~250 ms).**

In [74]:
f.close()
!ls -lh ctest-blosc.h5

-rw-rw-r-- 1 francesc francesc 7.7M Oct 21 15:27 ctest-blosc.h5


As you see, the compression ratio is quite the same than with zlib.

## Exercise

PyTables comes with support for different compressors, namely 'zlib' (the default), 'bzip2' and 'blosc:X' where X is a codec can be one of 'blosclz', 'lz4', 'lz4hc', 'snappy', 'zlib' (and 'zstd' from PyTables 3.3 on).  Based on the example above, do a small study on which ones work best.

* Which one compresses best?
* Which one compresses faster?
* Which one shows the best balance?

### Solution

In [75]:
na = np.linspace(0, 1, 1e7).reshape(10000,1000)
for complib in ['blosc:blosclz', 'blosc:lz4', 'blosc:lz4hc',
                'blosc:snappy', 'blosc:zlib', 'blosc:zstd',
                'zlib', 'bzip2']:
    with tb.open_file("compression_study.h5", "w") as f:
        print("complib:", complib)
        filters = tb.Filters(complib=complib, complevel=9)
        ca = f.create_carray(f.root, 'carray', tb.Float64Atom(), (10000,1000),
                            filters=filters)
        %time ca[:] = na
    !ls -lh compression_study.h5

complib: blosc:blosclz
CPU times: user 72 ms, sys: 0 ns, total: 72 ms
Wall time: 70.7 ms
-rw-rw-r-- 1 francesc francesc 8.6M Oct 21 15:27 compression_study.h5
complib: blosc:lz4
CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 53.5 ms
-rw-rw-r-- 1 francesc francesc 7.7M Oct 21 15:27 compression_study.h5
complib: blosc:lz4hc
CPU times: user 268 ms, sys: 0 ns, total: 268 ms
Wall time: 264 ms
-rw-rw-r-- 1 francesc francesc 5.6M Oct 21 15:27 compression_study.h5
complib: blosc:snappy
CPU times: user 56 ms, sys: 4 ms, total: 60 ms
Wall time: 62.7 ms
-rw-rw-r-- 1 francesc francesc 11M Oct 21 15:27 compression_study.h5
complib: blosc:zlib
CPU times: user 508 ms, sys: 8 ms, total: 516 ms
Wall time: 518 ms
-rw-rw-r-- 1 francesc francesc 4.6M Oct 21 15:27 compression_study.h5
complib: blosc:zstd
CPU times: user 10.4 s, sys: 0 ns, total: 10.4 s
Wall time: 10.4 s
-rw-rw-r-- 1 francesc francesc 2.7M Oct 21 15:27 compression_study.h5
complib: zlib
CPU times: user 532 ms, sys: 0 ns, total: 5

In this case the one that achieves the best compression ratio is `blosc:ztsd` whereas `blosc:lz4` is the fastest.  Also, `blosc:lz4` shows a pretty good balance between speed and compression ratio.

# Specifying chunk size

When creating a chunked dataset, a chunksize is chosen automatically based on some heuristics.  However, you may want to specify your own chunksize and see the best for you:

In [76]:
na = np.linspace(0, 1, 1e7).reshape(10000,1000)
for size in range(1, 10):
    with tb.open_file("chunk_study.h5", "w") as f:
        chunkshape = (size*100, 1000)
        print("chunkshape:", chunkshape)
        filters = tb.Filters(complib="blosc:lz4", complevel=9)
        ca = f.create_carray(f.root, 'carray', tb.Float64Atom(), (10000,1000),
                            filters=filters, chunkshape=chunkshape)
        %time ca[:] = na
    !ls -lh chunk_study.h5

chunkshape: (100, 1000)
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 40.7 ms
-rw-rw-r-- 1 francesc francesc 4.4M Oct 21 15:27 chunk_study.h5
chunkshape: (200, 1000)
CPU times: user 40 ms, sys: 0 ns, total: 40 ms
Wall time: 38.9 ms
-rw-rw-r-- 1 francesc francesc 4.2M Oct 21 15:27 chunk_study.h5
chunkshape: (300, 1000)
CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 38.9 ms
-rw-rw-r-- 1 francesc francesc 4.1M Oct 21 15:27 chunk_study.h5
chunkshape: (400, 1000)
CPU times: user 36 ms, sys: 12 ms, total: 48 ms
Wall time: 47.6 ms
-rw-rw-r-- 1 francesc francesc 4.1M Oct 21 15:27 chunk_study.h5
chunkshape: (500, 1000)
CPU times: user 40 ms, sys: 12 ms, total: 52 ms
Wall time: 50.3 ms
-rw-rw-r-- 1 francesc francesc 4.0M Oct 21 15:27 chunk_study.h5
chunkshape: (600, 1000)
CPU times: user 52 ms, sys: 0 ns, total: 52 ms
Wall time: 49.9 ms
-rw-rw-r-- 1 francesc francesc 4.1M Oct 21 15:27 chunk_study.h5
chunkshape: (700, 1000)
CPU times: user 44 ms, sys: 8 ms, total: 52 ms
Wal

Looks like a size of `200*1000*8` = ~1.6 MB gives the best speed for writing.

# Using ptrepack

You can use the 'ptrepack' utility to copy HDF5 whole files (or only parts) and change different parameters during the copy process.

In [77]:
!ptrepack -o --complib zlib --complevel 1 ctest.h5 ctest-repacked-zlib.h5

In [78]:
!ptrepack -o --complib blosc:lz4 --complevel 9 ctest.h5 ctest-repacked-blosc-lz4.h5

In [79]:
!ptrepack -o --complib blosc:zstd --complevel 5 ctest.h5 ctest-repacked-blosc-zstd.h5

In [80]:
!ptrepack -o --complib blosc:zstd --complevel 9 --chunkshape '(1000,1000)' ctest.h5 ctest-repacked-blosc-zstd-chunkshape.h5

In [81]:
!ls -lh ctest*.h5

-rw-rw-r-- 1 francesc francesc 7.7M Oct 21 15:27 ctest-blosc.h5
-rw-rw-r-- 1 francesc francesc 4.0M Oct 21 13:05 ctest-repacked-blosc-chunkshape.h5
-rw-rw-r-- 1 francesc francesc 7.7M Oct 21 15:27 ctest-repacked-blosc-lz4.h5
-rw-rw-r-- 1 francesc francesc 560K Oct 21 15:28 ctest-repacked-blosc-zstd-chunkshape.h5
-rw-rw-r-- 1 francesc francesc 5.4M Oct 21 15:27 ctest-repacked-blosc-zstd.h5
-rw-rw-r-- 1 francesc francesc 7.7M Oct 21 12:43 ctest-repacked-blosc.h5
-rw-rw-r-- 1 francesc francesc 6.8M Oct 21 15:27 ctest-repacked-zlib.h5
-rw-rw-r-- 1 francesc francesc 6.8M Oct 21 15:27 ctest-zlib.h5
-rw-rw-r-- 1 francesc francesc  77M Oct 21 15:27 ctest.h5


In [82]:
!ptrepack -h

usage: ptrepack [-h] [-v] [-o] [-R RANGE] [--non-recursive]
                [--dest-title TITLE] [--dont-create-sysattrs]
                [--dont-copy-userattrs] [--overwrite-nodes]
                [--complevel COMPLEVEL]
                [--complib {zlib,lzo,bzip2,blosc,blosc:blosclz,blosc:lz4,blosc:lz4hc,blosc:snappy,blosc:zlib,blosc:zstd}]
                [--shuffle {0,1}] [--bitshuffle {0,1}] [--fletcher32 {0,1}]
                [--keep-source-filters] [--chunkshape CHUNKSHAPE]
                [--upgrade-flavors] [--dont-regenerate-old-indexes]
                [--sortby COLUMN] [--checkCSI] [--propindexes]
                sourcefile:sourcegroup destfile:destgroup

This utility is very powerful and lets you copy any leaf, group or complete
subtree into another file. During the copy process you are allowed to change
the filter properties if you want so. Also, in the case of duplicated
pathnames, you can decide if you want to overwrite already existing nodes on
the destination file. Ge

# Queries in Table objects

In [83]:
# The description for the tabular data
class TabularData(tb.IsDescription):
    col1 = tb.StringCol(200)
    col2 = tb.IntCol()
    col3 = tb.FloatCol()

In [84]:
# Open a file and create the Table container
f = tb.open_file('atable.h5', 'w')
t = f.create_table(f.root, 'table', TabularData, 'table title',
                   filters=tb.Filters(9, 'blosc'))

In [85]:
t

/table (Table(0,), shuffle, blosc(9)) 'table title'
  description := {
  "col1": StringCol(itemsize=200, shape=(), dflt=b'', pos=0),
  "col2": Int32Col(shape=(), dflt=0, pos=1),
  "col3": Float64Col(shape=(), dflt=0.0, pos=2)}
  byteorder := 'little'
  chunkshape := (309,)

In [86]:
%%time
#  Fill the table with some 1 million rows
r = t.row
for i in range(1000*1000):
    r['col1'] = str(i)
    r['col2'] = i + 1
    r['col3'] = i * (i + 1)
    r.append()
t.flush()

CPU times: user 788 ms, sys: 12 ms, total: 800 ms
Wall time: 800 ms


In [87]:
t

/table (Table(1000000,), shuffle, blosc(9)) 'table title'
  description := {
  "col1": StringCol(itemsize=200, shape=(), dflt=b'', pos=0),
  "col2": Int32Col(shape=(), dflt=0, pos=1),
  "col3": Float64Col(shape=(), dflt=0.0, pos=2)}
  byteorder := 'little'
  chunkshape := (309,)

In [88]:
# Size on disk
!ls -lh atable.h5

-rw-rw-r-- 1 francesc francesc 4.9M Oct 21 15:28 atable.h5


In [89]:
# Real size
np.prod(t.shape) * t.dtype.itemsize / 2**20.

202.178955078125

In [90]:
# Do a query (regular)
%time [r['col1'] for r in t if r['col2'] < 5]

CPU times: user 120 ms, sys: 0 ns, total: 120 ms
Wall time: 120 ms


[b'0', b'1', b'2', b'3']

In [91]:
# Repeat the query, but using in-kernel method
%time [r['col1'] for r in t.where('col2 < 5')]

CPU times: user 84 ms, sys: 12 ms, total: 96 ms
Wall time: 79 ms


[b'0', b'1', b'2', b'3']

In [92]:
# Performing complex conditions (regular query)
%time [r['col1'] for r in t if r['col2'] < 5 and r['col3'] < 10]

CPU times: user 116 ms, sys: 0 ns, total: 116 ms
Wall time: 115 ms


[b'0', b'1', b'2']

In [93]:
# Complex, in-kernel queries
%time [r['col1'] for r in t.where('(col2 < 5) & (col3 < 10)')]

CPU times: user 80 ms, sys: 24 ms, total: 104 ms
Wall time: 81.8 ms


[b'0', b'1', b'2']

In [94]:
# Get a structured array out of disk
sa = t[:]
sa

array([(b'0', 1, 0.0), (b'1', 2, 2.0), (b'2', 3, 6.0), ...,
       (b'999997', 999998, 999995000006.0),
       (b'999998', 999999, 999997000002.0),
       (b'999999', 1000000, 999999000000.0)], 
      dtype=[('col1', 'S200'), ('col2', '<i4'), ('col3', '<f8')])

In [95]:
# Perform the query in-memory using pure NumPy machinery 
%time sa[((sa['col2'] < 5) & (sa['col3'] < 10))]['col1']

CPU times: user 20 ms, sys: 0 ns, total: 20 ms
Wall time: 16.1 ms


array([b'0', b'1', b'2'], 
      dtype='|S200')

In [96]:
# Create an index for the on-disk table
%time t.cols.col2.create_csindex()

CPU times: user 296 ms, sys: 12 ms, total: 308 ms
Wall time: 306 ms


1000000

In [97]:
# Repeat the complex query (indexed)
%time [r['col1'] for r in t.where('(col2 < 5) & (col3 < 10)')]

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.55 ms


[b'0', b'1', b'2']

Indexing normally offers the best speed for doing queries.

In [98]:
f.close()

# Exercise

Open the 'ic_dst...' file in the data/ directory:

In [99]:
f = tb.open_file("../data/ic_dst_NEXT_v0_08_02_Kr_ACTIVE_0_0_5bar_MCRD_10000.root.h5")

OSError: ``../data/ic_dst_NEXT_v0_08_02_Kr_ACTIVE_0_0_5bar_MCRD_10000.root.h5`` does not exist

In [ ]:
print(f)


* Determine the chunksize of the /MLR/mau and /RD/pmtrwf datasets

* Copy them to another (new) HDF5 file using different chunksizes and compressors.  Determine the ones that offers best ratio and speed. (use ptrepack).

* Use the /TWF/TWF and /Sensors/DataSiPM and do some small analysis (e.g. plotting the times for TWF, or query them based on some conditions that make sense).

### Solution